# INCONSISTENT DATA ENTRY

In [1]:
# MODULES WE WILL USE
import pandas as pd 
import numpy as np 

In [5]:
# HELPFUL MODULES
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

C:\Users\user\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
# we need to pip3 install fuzzywuzzy in anaconda cm !!

In [8]:
# read in all our data
ks = pd.read_csv('ks-projects-201801.csv')
ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [9]:
# set seed for reproducibility
np.random.seed(0)

In [10]:
# clean up the category column to make sure there is not data entry inconsistencies
# get all the values in category coloumn 
category = ks['category'].unique()

In [11]:
# sort them alphabetically and then take a closer look
category.sort()
category

array(['3D Printing', 'Academic', 'Accessories', 'Action', 'Animals',
       'Animation', 'Anthologies', 'Apparel', 'Apps', 'Architecture',
       'Art', 'Art Books', 'Audio', 'Bacon', 'Blues', 'Calendars',
       'Camera Equipment', 'Candles', 'Ceramics', "Children's Books",
       'Childrenswear', 'Chiptune', 'Civic Design', 'Classical Music',
       'Comedy', 'Comic Books', 'Comics', 'Community Gardens',
       'Conceptual Art', 'Cookbooks', 'Country & Folk', 'Couture',
       'Crafts', 'Crochet', 'DIY', 'DIY Electronics', 'Dance', 'Design',
       'Digital Art', 'Documentary', 'Drama', 'Drinks',
       'Electronic Music', 'Embroidery', 'Events', 'Experimental',
       'Fabrication Tools', 'Faith', 'Family', 'Fantasy',
       "Farmer's Markets", 'Farms', 'Fashion', 'Festivals', 'Fiction',
       'Film & Video', 'Fine Art', 'Flight', 'Food', 'Food Trucks',
       'Footwear', 'Gadgets', 'Games', 'Gaming Hardware', 'Glass',
       'Graphic Design', 'Graphic Novels', 'Hardware', 'Hip-Ho

In [12]:
'''The first thing I'm going to do is make everything lower case (I can change it back at the end if I like) and remove any white spaces at the beginning and end of cells. Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this.'''

"The first thing I'm going to do is make everything lower case (I can change it back at the end if I like) and remove any white spaces at the beginning and end of cells. Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this."

In [13]:
# convert to lower case
ks['category'] = ks['category'].str.lower()

# remove the trailing white spaces
ks['category'] = ks['category'].str.strip()
ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,narrative film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,narrative film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,film & video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [14]:
# use fuzzy matching to correct inconsistent data entry

In [15]:
# get all the unique values in the category column
categories = ks['category'].unique()

# sort them alphabetically and than take a closer look
categories.sort()
categories

array(['3d printing', 'academic', 'accessories', 'action', 'animals',
       'animation', 'anthologies', 'apparel', 'apps', 'architecture',
       'art', 'art books', 'audio', 'bacon', 'blues', 'calendars',
       'camera equipment', 'candles', 'ceramics', "children's books",
       'childrenswear', 'chiptune', 'civic design', 'classical music',
       'comedy', 'comic books', 'comics', 'community gardens',
       'conceptual art', 'cookbooks', 'country & folk', 'couture',
       'crafts', 'crochet', 'dance', 'design', 'digital art', 'diy',
       'diy electronics', 'documentary', 'drama', 'drinks',
       'electronic music', 'embroidery', 'events', 'experimental',
       'fabrication tools', 'faith', 'family', 'fantasy',
       "farmer's markets", 'farms', 'fashion', 'festivals', 'fiction',
       'film & video', 'fine art', 'flight', 'food', 'food trucks',
       'footwear', 'gadgets', 'games', 'gaming hardware', 'glass',
       'graphic design', 'graphic novels', 'hardware', 'hip-ho

In [16]:
# We're going to use the fuzzywuzzy package to help identify which strings are closest to each other

 Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another

In [17]:
#Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100

In [20]:
# get the top 10 closest matches to 'games'
matches = fuzzywuzzy.process.extract('games', categories, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('games', 100),
 ('gadgets', 67),
 ('live games', 67),
 ('video games', 62),
 ('farms', 60),
 ('glass', 60),
 ('mobile games', 59),
 ('places', 55),
 ('spaces', 55),
 ('tabletop games', 53)]

In [27]:
# function to replace rows in the provided column of the provided data frame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 67):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, scorer= fuzzywuzzy.fuzz.token_sort_ratio)
    
    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    
    # replace all rows with close mathces with the input matches
    
    df.loc[rows_with_matches, column] = string_to_match

#let us know the function's done
print("all done!")

all done!


In [29]:
# use the function we just wrote to replace close matches to 'games' with 'games'
replace_matches_in_column(df=ks, column='category', string_to_match= 'games')

and now let's check the unique values in our 'category' column  again and make sure 
we've tidied up 'games ' correctly

In [30]:
# get all the values in the 'category' column
categoriese = ks['category'].unique()

# sort them alphabtically and then take a closer look 
categories.sort()
categories

array(['3d printing', 'academic', 'accessories', 'action', 'animals',
       'animation', 'anthologies', 'apparel', 'apps', 'architecture',
       'art', 'art books', 'audio', 'bacon', 'blues', 'calendars',
       'camera equipment', 'candles', 'ceramics', "children's books",
       'childrenswear', 'chiptune', 'civic design', 'classical music',
       'comedy', 'comic books', 'comics', 'community gardens',
       'conceptual art', 'cookbooks', 'country & folk', 'couture',
       'crafts', 'crochet', 'dance', 'design', 'digital art', 'diy',
       'diy electronics', 'documentary', 'drama', 'drinks',
       'electronic music', 'embroidery', 'events', 'experimental',
       'fabrication tools', 'faith', 'family', 'fantasy',
       "farmer's markets", 'farms', 'fashion', 'festivals', 'fiction',
       'film & video', 'fine art', 'flight', 'food', 'food trucks',
       'footwear', 'gadgets', 'games', 'gaming hardware', 'glass',
       'graphic design', 'graphic novels', 'hardware', 'hip-ho

In [35]:
matches

[('games', 100),
 ('gadgets', 67),
 ('live games', 67),
 ('video games', 62),
 ('farms', 60),
 ('glass', 60),
 ('mobile games', 59),
 ('places', 55),
 ('spaces', 55),
 ('tabletop games', 53)]

NameError: name 'close_matches' is not defined